In [ ]:
%matplotlib inline
import os, shutil
import sys
sys.path.append("..")
import numpy as np
from IPython.display import Image
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import flopy as flopy
import pyemu

In [ ]:
import freyberg_setup as fs
fs.setup_pest_gr()
working_dir = fs.WORKING_DIR_GR
pst_name = fs.PST_NAME_GR

In [ ]:
fs.plot_model(working_dir, pst_name)

In [ ]:
m = flopy.modflow.Modflow.load(fs.MODEL_NAM,model_ws=working_dir,load_only=["upw"],check=False)

In [ ]:
v = pyemu.geostats.ExpVario(1.0,a=200,anisotropy=1.0,bearing=45)
struct = pyemu.geostats.GeoStruct(variograms=v)
arr_dict = {"test":np.ones((m.nrow,m.ncol))}

In [ ]:
bd = pyemu.helpers.kl_setup(num_eig=800,sr=m.sr,struct=struct,array_dict=arr_dict,basis_file="basis.jco")

In [ ]:
basis = pyemu.Matrix.from_binary("basis.jco").to_dataframe().T

In [ ]:
i = basis.index.map(lambda x: int(x[1:5]))
j = basis.index.map(lambda x: int(x[-4:]))

In [ ]:
def plot_eig_arr(eig):
    arr = np.zeros((m.nrow,m.ncol))
    arr[i,j] = basis.iloc[:,eig]
    mm = flopy.plot.ModelMap(model=m)
    mm.plot_array(arr)
    mm.plot_ibound()
    #mm = plt.imshow(arr)
    return mm

In [ ]:
mm = plot_eig_arr(2)

In [ ]:
#interact(plot_eig_arr, eig=widgets.IntSlider(min=0,max=799,step=1,value=10));
interact(plot_eig_arr, eig=widgets.IntText(description="eig comp", continuous_update=True));

### Now lets look at how the KL reconstruction looks for a nasty zone array...

In [ ]:
arr = np.loadtxt(os.path.join("..","..","models","Freyberg","Freyberg_Truth","hk.zones"))
mm = flopy.plot.ModelMap(model=m)
mm.plot_array(arr)
mm.plot_ibound()

In [ ]:
basis_arr = np.array(basis.values)
flat_arr = np.atleast_2d(arr.flatten()).transpose()

In [ ]:
def plot_eig_arr(eig):
    basis_eig = basis_arr[:,:eig+1].transpose()
    factors = np.dot(basis_eig,flat_arr).transpose()
    factors = np.dot(factors,basis_eig).reshape(arr.shape)
    mm = flopy.plot.ModelMap(model=m)
    mm.plot_array(factors)
    mm.plot_ibound()
    return mm

In [ ]:
interact(plot_eig_arr, eig=widgets.IntText(description="eig comp", continuous_update=True));